In [0]:
!pip install git+https://github.com/Theano/Theano
!pip install git+https://github.com/Lasagne/Lasagne
!pip install git+https://github.com/tariqdaouda/Mariana

  Cloning https://github.com/Theano/Theano to /tmp/pip-req-build-x2sn61nl
  Running command git clone -q https://github.com/Theano/Theano /tmp/pip-req-build-x2sn61nl
  Created wheel for Theano: filename=Theano-1.0.4+51.gf1e4ec47c-cp36-none-any.whl size=2667983 sha256=d9bb0f21d987ca3f7341b2bb39a4c73faf8768496246dae224dc99f4545e3a6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-u00wyh4f/wheels/64/f2/f4/6b1f50baf18aca2eab5d9b5a431b90e3d8be4711c8f7457eb7
Successfully built Theano
  Cloning https://github.com/Lasagne/Lasagne to /tmp/pip-req-build-kmxaa20r
  Running command git clone -q https://github.com/Lasagne/Lasagne /tmp/pip-req-build-kmxaa20r
  Created wheel for Lasagne: filename=Lasagne-0.2.dev1-cp36-none-any.whl size=122797 sha256=4271e2d2c25b7e1905618f870332cbb469173fcf62ea740d85d9d8550ee4bb23
  Stored in directory: /tmp/pip-ephem-wheel-cache-4s50hsk7/wheels/c4/20/90/9f7242c381402829c5918261e3eb51a87bc1d8521456749b57
Successfully built Lasagne
  Cloning https://github.com/tariqd

In [0]:

import math as m
import numpy as np
np.random.seed(123)
import scipy.io
from sklearn.decomposition import PCA

def cart2sph(x, y, z):
    x2_y2 = x**2 + y**2
    r = m.sqrt(x2_y2 + z**2)                    
    elev = m.atan2(z, m.sqrt(x2_y2))            
    az = m.atan2(y, x)                        
    return r, elev, az


def pol2cart(theta, rho):
    return rho * m.cos(theta), rho * m.sin(theta)


def augment_EEG(data, stdMult, pca=False, n_components=2):
    
    augData = np.zeros(data.shape)
    if pca:
        pca = PCA(n_components=n_components)
        pca.fit(data)
        components = pca.components_
        variances = pca.explained_variance_ratio_
        coeffs = np.random.normal(scale=stdMult, size=pca.n_components) * variances
        for s, sample in enumerate(data):
            augData[s, :] = sample + (components * coeffs.reshape((n_components, -1))).sum(axis=0)
    else:
        for f, feat in enumerate(data.transpose()):
            augData[:, f] = feat + np.random.normal(scale=stdMult*np.std(feat), size=feat.size)
    return augData


def augment_EEG_image(image, std_mult, pca=False, n_components=2):
    
    augData = np.zeros((data.shape[0], data.shape[1], data.shape[2] * data.shape[3]))
    for c in xrange(image.shape[1]):
        reshData = np.reshape(data['featMat'][:, c, :, :], (data['featMat'].shape[0], -1))
        if pca:
            augData[:, c, :] = augment_EEG(reshData, std_mult, pca=True, n_components=n_components)
        else:
            augData[:, c, :] = augment_EEG(reshData, std_mult, pca=False)
    return np.reshape(augData, data['featMat'].shape)


def load_data(data_file):
   
    print("Loading data from %s" % (data_file))

    dataMat = scipy.io.loadmat(data_file, mat_dtype=True)

    print("Data loading complete. Shape is %r" % (dataMat['featMat'].shape,))
    return dataMat['features'][:, :-1], dataMat['features'][:, -1] - 1   

def reformatInput(data, labels, indices):
   
    trainIndices = indices[0][len(indices[1]):]
    validIndices = indices[0][:len(indices[1])]
    testIndices = indices[1]
    
    if data.ndim == 4:
        return [(data[trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
                (data[validIndices], np.squeeze(labels[validIndices]).astype(np.int32)),
                (data[testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]
    elif data.ndim == 5:
        return [(data[:, trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
                (data[:, validIndices], np.squeeze(labels[validIndices]).astype(np.int32)),
                (data[:, testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]


if __name__ == '__main__':
    data = np.random.normal(size=(100, 10))
    print('Original: {0}'.format(data))   
    print('Augmented: {0}'.format(augment_EEG(data, 0.1, pca=True)))

Original: [[-1.08563060e+00  9.97345447e-01  2.82978498e-01 -1.50629471e+00
  -5.78600252e-01  1.65143654e+00 -2.42667924e+00 -4.28912629e-01
   1.26593626e+00 -8.66740402e-01]
 [-6.78886152e-01 -9.47089689e-02  1.49138963e+00 -6.38901997e-01
  -4.43981960e-01 -4.34351276e-01  2.20593008e+00  2.18678609e+00
   1.00405390e+00  3.86186399e-01]
 [ 7.37368576e-01  1.49073203e+00 -9.35833868e-01  1.17582904e+00
  -1.25388067e+00 -6.37751502e-01  9.07105196e-01 -1.42868070e+00
  -1.40068720e-01 -8.61754896e-01]
 [-2.55619371e-01 -2.79858911e+00 -1.77153310e+00 -6.99877235e-01
   9.27462432e-01 -1.73635683e-01  2.84591590e-03  6.88222711e-01
  -8.79536343e-01  2.83627324e-01]
 [-8.05366518e-01 -1.72766949e+00 -3.90899794e-01  5.73805862e-01
   3.38589051e-01 -1.18304945e-02  2.39236527e+00  4.12912160e-01
   9.78736006e-01  2.23814334e+00]
 [-1.29408532e+00 -1.03878821e+00  1.74371223e+00 -7.98062735e-01
   2.96832303e-02  1.06931597e+00  8.90706391e-01  1.75488618e+00
   1.49564414e+00  1.06

In [0]:
from __future__ import print_function
import time

import numpy as np
np.random.seed(1234)
from functools import reduce
import math as m

import scipy.io
import theano
import theano.tensor as T

from scipy.interpolate import griddata
from sklearn.preprocessing import scale

import lasagne
from lasagne.regularization import regularize_layer_params, regularize_network_params, l1, l2
from lasagne.layers import Conv2DLayer, MaxPool2DLayer, InputLayer
from lasagne.layers import DenseLayer, ElemwiseMergeLayer, FlattenLayer
from lasagne.layers import ConcatLayer, ReshapeLayer, get_output_shape
from lasagne.layers import Conv1DLayer, DimshuffleLayer, LSTMLayer, SliceLayer


def azim_proj(pos):
    
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, m.pi / 2 - elev)


def gen_images(locs, features, n_gridpoints, normalize=True, augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
    
    feat_array_temp = []
    nElectrodes = locs.shape[0]    

    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1]//nElectrodes
    for c in range(n_colors):
        feat_array_temp.append(features[:, c * nElectrodes : nElectrodes * (c+1)])
    if augment:
        if pca:
            for c in range(n_colors):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=True, n_components=n_components)
        else:
            for c in range(n_colors):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=False, n_components=n_components)
    n_samples = features.shape[0]

    grid_x, grid_y = np.mgrid[min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j, min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j]
    temp_interp = []
    for c in range(n_colors):
        temp_interp.append(np.zeros([n_samples, n_gridpoints, n_gridpoints]))

    if edgeless:
        min_x, min_y = np.min(locs, axis=0)
        max_x, max_y = np.max(locs, axis=0)
        locs = np.append(locs, np.array([[min_x, min_y], [min_x, max_y], [max_x, min_y], [max_x, max_y]]), axis=0)
        for c in range(n_colors):
            feat_array_temp[c] = np.append(feat_array_temp[c], np.zeros((n_samples, 4)), axis=1)

    
    for i in range(n_samples):
        for c in range(n_colors):
            temp_interp[c][i, :, :] = griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),
                                               method='cubic', fill_value=np.nan)
        print('Interpolating {0}/{1}\r'.format(i + 1, n_samples), end='\r')

    for c in range(n_colors):
        if normalize:
            temp_interp[c][~np.isnan(temp_interp[c])] = \
                scale(temp_interp[c][~np.isnan(temp_interp[c])])
        temp_interp[c] = np.nan_to_num(temp_interp[c])
    return np.swapaxes(np.asarray(temp_interp), 0, 1)     

def build_cnn(input_var=None, w_init=None, n_layers=(4, 2, 1), n_filters_first=32, imsize=32, n_colors=3):
    
    weights = []        
    count = 0
    
    if w_init is None:
        w_init = [lasagne.init.GlorotUniform()] * sum(n_layers)
    network = InputLayer(shape=(None, n_colors, imsize, imsize),
                                        input_var=input_var)
    for i, s in enumerate(n_layers):
        for l in range(s):
            network = Conv2DLayer(network, num_filters=n_filters_first * (2 ** i), filter_size=(3, 3),
                          W=w_init[count], pad='same')
            count += 1
            weights.append(network.W)
        network = MaxPool2DLayer(network, pool_size=(2, 2))
    return network, weights


def build_convpool_max(input_vars, nb_classes, imsize=32, n_colors=3, n_timewin=7):
    convnets = []
    w_init = None
    
    for i in range(n_timewin):
        if i == 0:
            convnet, w_init = build_cnn(input_vars[i], imsize=imsize, n_colors=n_colors)
        else:
            convnet, _ = build_cnn(input_vars[i], w_init=w_init, imsize=imsize, n_colors=n_colors)
        convnets.append(convnet)
    
    convpool = ElemwiseMergeLayer(convnets, theano.tensor.maximum)
    
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=512, nonlinearity=lasagne.nonlinearities.rectify)
    
    convpool = lasagne.layers.DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=nb_classes, nonlinearity=lasagne.nonlinearities.softmax)
    return convpool


def build_convpool_conv1d(input_vars, nb_classes, imsize=32, n_colors=3, n_timewin=7):
    
    convnets = []
    w_init = None
    
    for i in range(n_timewin):
        if i == 0:
            convnet, w_init = build_cnn(input_vars[i], imsize=imsize, n_colors=n_colors)
        else:
            convnet, _ = build_cnn(input_vars[i], w_init=w_init, imsize=imsize, n_colors=n_colors)
        convnets.append(FlattenLayer(convnet))
    convpool = ConcatLayer(convnets)
    convpool = ReshapeLayer(convpool, ([0], n_timewin, get_output_shape(convnets[0])[1]))
    convpool = DimshuffleLayer(convpool, (0, 2, 1))
    convpool = Conv1DLayer(convpool, 64, 3)
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=512, nonlinearity=lasagne.nonlinearities.rectify)
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=nb_classes, nonlinearity=lasagne.nonlinearities.softmax)
    return convpool


def build_convpool_lstm(input_vars, nb_classes, grad_clip=110, imsize=32, n_colors=3, n_timewin=7):
    
    convnets = []
    w_init = None
    for i in range(n_timewin):
        if i == 0:
            convnet, w_init = build_cnn(input_vars[i], imsize=imsize, n_colors=n_colors)
        else:
            convnet, _ = build_cnn(input_vars[i], w_init=w_init, imsize=imsize, n_colors=n_colors)
        convnets.append(FlattenLayer(convnet))
    
    convpool = ConcatLayer(convnets)
    convpool = ReshapeLayer(convpool, ([0], n_timewin, get_output_shape(convnets[0])[1]))
    convpool = LSTMLayer(convpool, num_units=128, grad_clipping=grad_clip,
        nonlinearity=lasagne.nonlinearities.tanh)
    
    convpool = SliceLayer(convpool, -1, 1)     
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=256, nonlinearity=lasagne.nonlinearities.rectify)
    convpool = DenseLayer(lasagne.layers.dropout(convpool, p=.5),
            num_units=nb_classes, nonlinearity=lasagne.nonlinearities.softmax)
    return convpool


def build_convpool_mix(input_vars, nb_classes, grad_clip=110, imsize=32, n_colors=3, n_timewin=7):
    
    convnets = []
    w_init = None
    for i in range(n_timewin):
        if i == 0:
            convnet, w_init = build_cnn(input_vars[i], imsize=imsize, n_colors=n_colors)
        else:
            convnet, _ = build_cnn(input_vars[i], w_init=w_init, imsize=imsize, n_colors=n_colors)
        convnets.append(FlattenLayer(convnet))
    convpool = ConcatLayer(convnets)
    convpool = ReshapeLayer(convpool, ([0], n_timewin, get_output_shape(convnets[0])[1]))
    reformConvpool = DimshuffleLayer(convpool, (0, 2, 1))
    conv_out = Conv1DLayer(reformConvpool, 64, 3)
    conv_out = FlattenLayer(conv_out)
    lstm = LSTMLayer(convpool, num_units=128, grad_clipping=grad_clip,
        nonlinearity=lasagne.nonlinearities.tanh)
    lstm_out = SliceLayer(lstm, -1, 1)
    dense_input = ConcatLayer([conv_out, lstm_out])
    convpool = DenseLayer(lasagne.layers.dropout(dense_input, p=.5),
            num_units=512, nonlinearity=lasagne.nonlinearities.rectify)
    convpool = DenseLayer(convpool,
            num_units=nb_classes, nonlinearity=lasagne.nonlinearities.softmax)
    return convpool


def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    
    if inputs.ndim == 4:
        input_len = inputs.shape[0]
    elif inputs.ndim == 5:
        input_len = inputs.shape[1]
    assert input_len == len(targets)
    if shuffle:
        indices = np.arange(input_len)
        np.random.shuffle(indices)
    for start_idx in range(0, input_len, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        if inputs.ndim == 4:
            yield inputs[excerpt], targets[excerpt]
        elif inputs.ndim == 5:
            yield inputs[:, excerpt], targets[excerpt]


def train(images, labels, fold, model_type, batch_size=32, num_epochs=5):
    
    print('train')
    num_classes = len(np.unique(labels))
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = reformatInput(images, labels, fold)
    X_train = X_train.astype("float32", casting='unsafe')
    X_val = X_val.astype("float32", casting='unsafe')
    X_test = X_test.astype("float32", casting='unsafe')
    input_var = T.TensorType('floatX', ((False,) * 4))()
    target_var = T.ivector('targets')
    print("Building model and compiling functions...")
    if model_type == '1dconv':
        network = build_convpool_conv1d(input_var, num_classes)
    elif model_type == 'maxpool':
        network = build_convpool_max(input_var, num_classes)
    elif model_type == 'lstm':
        network = build_convpool_lstm(input_var, num_classes, 100)
    elif model_type == 'mix':
        network = build_convpool_mix(input_var, num_classes, 100)
    elif model_type == 'cnn':
        input_var = T.tensor4('inputs')
        network, _ = build_cnn(input_var)
        network = DenseLayer(lasagne.layers.dropout(network, p=.5),
                             num_units=256,
                             nonlinearity=lasagne.nonlinearities.rectify)
        network = DenseLayer(lasagne.layers.dropout(network, p=.5),
                             num_units=num_classes,
                             nonlinearity=lasagne.nonlinearities.softmax)
    else:
        raise ValueError("Model not supported ['1dconv', 'maxpool', 'lstm', 'mix', 'cnn']")
  
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()
    reg_factor = 1e-4
    l2_penalty = regularize_network_params(network, l2) * reg_factor
    loss += l2_penalty

    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.adam(loss, params, learning_rate=0.001)

    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)
 
    train_fn = theano.function([input_var, target_var], loss, updates=updates)
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
    print("Starting training...")
    best_validation_accu = 0
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        print(9)
        for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=False):
            inputs, targets = batch

            inputs1=inputs
            inputs=inputs1[1,:,:,:,:]
            train_err += train_fn(inputs, targets)
            
            
            inputs=inputs1[2,:,:,:,:]
            train_err += train_fn(inputs, targets)

            
            inputs=inputs1[3,:,:,:,:]
            train_err += train_fn(inputs, targets)

            
            inputs=inputs1[4,:,:,:,:]
            train_err += train_fn(inputs, targets)

            
            inputs=inputs1[5,:,:,:,:]
            train_err += train_fn(inputs, targets)

            
            inputs=inputs1[6,:,:,:,:]
            train_err += train_fn(inputs, targets)

            
            inputs=inputs1[0,:,:,:,:]
            train_err += train_fn(inputs, targets)

            train_batches += 1
        val_err = 0
        print('01')

        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, batch_size, shuffle=False):
            inputs, targets = batch
            
            inputs1=inputs
            inputs=inputs1[1,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            
            inputs=inputs1[2,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            inputs=inputs1[3,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            inputs=inputs1[4,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            inputs=inputs1[5,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            inputs=inputs1[6,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc

            inputs=inputs1[0,:,:,:,:]
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            
            val_batches += 1

        av_train_err = train_err / train_batches
        av_train_err = av_train_err / 7

        av_val_err = val_err / val_batches
        av_val_acc = val_acc / val_batches

        av_val_err = av_val_err / 7
        av_val_acc = av_val_acc / 7
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(av_train_err))
        print("  validation loss:\t\t{:.6f}".format(av_val_err))
        print("  validation accuracy:\t\t{:.2f} %".format(av_val_acc * 100))
        if av_val_acc > best_validation_accu:
            best_validation_accu = av_val_acc
            test_err = 0
            print('00')
            test_acc = 0
            test_batches = 0
            for batch in iterate_minibatches(X_test, y_test, batch_size, shuffle=False):
                inputs, targets = batch
                
                inputs1=inputs
                inputs=inputs1[1,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                inputs=inputs1[2,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc
                
                inputs=inputs1[3,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                inputs=inputs1[4,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                inputs=inputs1[5,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                inputs=inputs1[6,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                inputs=inputs1[0,:,:,:,:]
                err, acc = val_fn(inputs, targets)
                test_err += err
                test_acc += acc

                test_batches += 1
            av_test_err = test_err / test_batches
            av_test_acc = test_acc / test_batches
            
            av_test_err = av_test_err / 7
            av_test_acc = av_test_acc / 7
            
            print("Final results:")
            print("  test loss:\t\t\t{:.6f}".format(av_test_err))
            print("  test accuracy:\t\t{:.2f} %".format(av_test_acc * 100))
            np.savez('weights_lasg_{0}'.format(model_type), *lasagne.layers.get_all_param_values(network))
    print('-'*50)
    print("Best validation accuracy:\t\t{:.2f} %".format(best_validation_accu * 100))
    print("Best test accuracy:\t\t{:.2f} %".format(av_test_acc * 100))
    return av_test_acc




In [0]:

if __name__ == '__main__':
    

    print('Loading data...')
    locs = scipy.io.loadmat('/content/sample_data/Neuroscan_locs_orig.mat')
    locs_3d = locs['A']
    locs_2d = []
    for e in locs_3d:
        locs_2d.append(azim_proj(e))

    feats = scipy.io.loadmat('/content/sample_data/FeatureMat_timeWin.mat')['features']
    print(feats[:, -1])
    b=np.squeeze(feats[:, -1])- 1
    print(b.shape)
    print(feats[:, -1].shape)
    subj_nums = np.squeeze(scipy.io.loadmat('/content/sample_data/trials_subNums.mat')['subjectNum'])
    fold_pairs = []
    for i in np.unique(subj_nums):
        ts = subj_nums == i               
        tr = np.squeeze(np.nonzero(np.bitwise_not(ts))) 
        ts = np.squeeze(np.nonzero(ts))    
        np.random.shuffle(tr)  
        np.random.shuffle(ts)
        fold_pairs.append((tr, ts))

    
    images_timewin = np.array([gen_images(np.array(locs_2d),
                                          feats[:, i * 192:(i + 1) * 192], 32, normalize=True) for i in
                               range(feats.shape[1]//192)
                               ])
    
    print('\n')
    print('Training the LSTM-CONV Model...')
    test_acc_mix = []
    test_acc_cnn = []
    for i in range(len(fold_pairs)):
        print('fold {0}/{1}'.format(i+1, len(fold_pairs)))
        
        a=train(images_timewin,b,  fold_pairs[i], 'cnn')
        
        test_acc_cnn.append(a)
    print('*' * 40)
    print('Average MIX test accuracy: {0}'.format(np.mean(test_acc_mix)*100))
    print('Average CNN test accuracy: {0}'.format(np.mean(test_acc_cnn) * 100))
    print('*' * 40)

    print('Done!')


Loading data...
[1. 1. 1. ... 4. 4. 4.]
(2670,)
(2670,)


Training the LSTM-CONV Model...
fold 1/13
train
Building model and compiling functions...
Starting training...
9
01
Epoch 1 of 5 took 436.442s
  training loss:		0.817328
  validation loss:		0.378874
  validation accuracy:		87.80 %
00
Final results:
  test loss:			1.763371
  test accuracy:		59.68 %
9
01
Epoch 2 of 5 took 445.839s
  training loss:		0.374214
  validation loss:		0.206743
  validation accuracy:		91.85 %
00
Final results:
  test loss:			1.451927
  test accuracy:		62.67 %
9
01
Epoch 3 of 5 took 441.556s
  training loss:		0.286212
  validation loss:		0.161100
  validation accuracy:		91.96 %
00
Final results:
  test loss:			1.258592
  test accuracy:		61.11 %
9
01
Epoch 4 of 5 took 435.177s
  training loss:		0.252533
  validation loss:		0.145060
  validation accuracy:		92.73 %
00
Final results:
  test loss:			1.472711
  test accuracy:		64.63 %
9
01
Epoch 5 of 5 took 438.288s
  training loss:		0.239961
  validation loss:		

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
